In [1]:
# %pip install -U git+https://github.com/graphcore-research/gfloat airium
# %load_ext autoreload
# %autoreload 2

<!-- # Copyright (c) 2024 Graphcore Ltd. All rights reserved. -->

# Making value tables

In this notebook, we generate value tables akin to those at [P3109](https://htmlpreview.github.io/?https://raw.githubusercontent.com/P3109/Public/main/Value%20Tables/html/index.html).

Thes tables comprise one-line summaries of each float value in the form
```text
Code Binary     = Exact binary E =  Float16 equivalent Float16 binary E    = Float Value
0x21 0_0100_001 = +0b1.001*2^-4  = 0_01011_0010000000 +0b1.0010000000*2^-4 = ~0.0703
```

In [2]:
import math
from gfloat import *
from gfloat.formats import *
import numpy as np
from IPython.display import HTML
import airium

## Define some helpers.

### Render with underscores separating s_e_m

E.g `0_1011_110`.  For formats with zero significand bits or zero exponent bits, we use `0_1011110_` or `0__10111110`.

In [3]:
def str_bits_with_underscores(fi, fv):
    signstr = f"{fv.signbit}_" if fi.is_signed else ""

    # 0_1011110_
    if fi.tSignificandBits == 0:
        return f"{signstr}{fv.exp:0{fi.expBits}b}_"

    # 0__1011110
    if fi.expBits == 0:
        return f"{signstr}{fv.significand:0{fi.tSignificandBits}b}"

    # 0_101_1110
    return f"{signstr}{fv.exp:0{fi.expBits}b}_{fv.significand:0{fi.tSignificandBits}b}"


fi = format_info_p3109(5, 3)
assert str_bits_with_underscores(fi, decode_float(fi, 0x09)) == "0_10_01"

fi = format_info_p3109(8, 1)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == "0_1000001_"

fi = format_info_p3109(8, 7)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == "0_1_000001"

### Render a binary16 value

Returns two strings, like this:
```
'0_00010_1010000000', '+0b1.1010000000*2^-13'
```

In [4]:
import struct


def b16_str(val) -> tuple[str, str]:
    """
    Represent VAL in binary16.

    If val does not convert exactly to binary16,
    returns "<Not16:{val}>"
    """
    with np.errstate(over="ignore"):
        b16 = np.float16(val)

    if float(b16) != val and np.isfinite(b16):
        # Finite, but not representable in float16
        return f"<Not16:{val}>", ""
    b16_int = struct.unpack("!H", struct.pack("!e", b16))[0]

    # bitstr is of the form 0_00000_1100000000
    s = f"{b16_int:016b}"
    e_str = s[1:6]
    m_str = s[6:]
    bitstr = f"{s[0]}_{e_str}_{m_str}"

    # pow2str is of the form '+0b0.1100000000*2^-15', or '' for nonfinite values
    e = int(e_str, 2) - 15
    m = int(m_str, 2)
    leading_bit = 0 if e == -15 else 1
    signstr = "-" if s[0] == "1" else "+"
    if np.isfinite(b16):
        pow2str = f"{signstr}0b{leading_bit}.{m:010b}*2^{e}"
    else:
        pow2str = ""
    return bitstr, pow2str


assert b16_str(13 * 2**-16) == ("0_00010_1010000000", "+0b1.1010000000*2^-13")

### Print one table row

In [5]:
def hex(fi, fv):
    return f"0x{fv.code:02x}"


def binary_pow2(fi, fv):
    """
    Create a string of the form
      0b1.01*2^3
    """
    finite_nonzero = np.isfinite(fv.fval) and fv.fval != 0

    if not finite_nonzero:
        return str(fv.fval)

    signstr = "-" if fv.signbit else "+"

    b = "0" if fv.fclass == FloatClass.SUBNORMAL else "1"

    return f"{signstr}0b{b}.{fv.significand:0{fi.tSignificandBits}b}*2^{fv.expval:<3}"


for fi in (format_info_p3109(8, 3), format_info_p3109(8, 1)):
    print("##", fi.name)
    codes = (0x00, 0x01, 0x07, 0x21, 0x40, 0x41, 0x7E, 0x7F, 0x80, 0x81, 0xE6, 0xFE, 0xFF)
    fvs = [decode_float(fi, i) for i in codes]
    for fv in fvs:
        print(
            hex(fi, fv),
            str_bits_with_underscores(fi, fv),
            binary_pow2(fi, fv),
            float_tilde_unless_roundtrip_str(fv.fval, width=8, d=4),
            sep=" = ",
        )

## p3109_k8p3se
0x00 = 0_00000_00 = 0.0 = 0.0
0x01 = 0_00000_01 = +0b0.01*2^-15 = ~7.629e-06
0x07 = 0_00001_11 = +0b1.11*2^-15 = ~5.341e-05
0x21 = 0_01000_01 = +0b1.01*2^-8  = ~0.0049
0x40 = 0_10000_00 = +0b1.00*2^0   = 1.0
0x41 = 0_10000_01 = +0b1.01*2^0   = 1.25
0x7e = 0_11111_10 = +0b1.10*2^15  = 49152.0
0x7f = 0_11111_11 = inf = inf
0x80 = 1_00000_00 = nan = nan
0x81 = 1_00000_01 = -0b0.01*2^-15 = ~-7.629e-06
0xe6 = 1_11001_10 = -0b1.10*2^9   = -768.0
0xfe = 1_11111_10 = -0b1.10*2^15  = -49152.0
0xff = 1_11111_11 = -inf = -inf
## p3109_k8p1se
0x00 = 0_0000000_ = 0.0 = 0.0
0x01 = 0_0000001_ = +0b1.0*2^-63 = ~1.084e-19
0x07 = 0_0000111_ = +0b1.0*2^-57 = ~6.939e-18
0x21 = 0_0100001_ = +0b1.0*2^-31 = ~4.657e-10
0x40 = 0_1000000_ = +0b1.0*2^0   = 1.0
0x41 = 0_1000001_ = +0b1.0*2^1   = 2.0
0x7e = 0_1111110_ = +0b1.0*2^62  = ~4.612e+18
0x7f = 0_1111111_ = inf = inf
0x80 = 1_0000000_ = nan = nan
0x81 = 1_0000001_ = -0b1.0*2^-63 = ~-1.084e-19
0xe6 = 1_1100110_ = -0b1.0*2^38  = ~-2.749e+11
0

## Make HTML table

In [6]:
def mktbl(
    airium_in, fi: FormatInfo, cols=4, skip_rows=None, dark=True, short=False, **kw
):
    # Make tables
    nvals = 2**fi.bits
    rows = nvals // cols

    # dark = True  # Used @media selector in CSS, but it doesn't work inside vscode
    css = ""

    def value_style(fv):
        if fv.fclass == FloatClass.SUBNORMAL:
            return "color: #0df" if dark else "color: #02b"

        if not (
            fv.fclass == FloatClass.NORMAL
            or fv.fclass == FloatClass.ZERO
            and not fv.signbit
        ):
            return "color: #d80" if dark else "color: #952"

        return ""

    title = f"FP{fi.k} Value Table, {fi.name}"
    a = airium_in

    with a.table(klass="zmktbl"):
        for i in range(0, rows):
            if skip_rows and i in skip_rows:
                if i - 1 not in skip_rows:
                    with a.tr(klass="zmktbl"):
                        for x in range(cols):
                            a.td(klass="zmktbl", _t="...", style="text-align: center;")
                continue
            if i > 0 and i % 16 == 0:
                # blank row
                trstyle = "blankrow"
            else:
                trstyle = "datarow"
            with a.tr(klass="zmktbl " + trstyle):
                for n in range(i, nvals, rows):
                    fv = decode_float(fi, n)
                    h = hex(fi, fv)
                    b = str_bits_with_underscores(fi, fv)
                    b2 = binary_pow2(fi, fv)
                    v = float_tilde_unless_roundtrip_str(fv.fval, **kw)
                    if short:
                        text = v
                    else:
                        text = " = ".join([h, b, b2, v])
                    with a.td(klass="zmktbl", style="text-align: left;"):
                        a.pre(klass="zmktbl", _t=text, style=value_style(fv))
    css += """
    table.zmktbl {
        margin: 0pt;
        border-collapse: collapse; 
    }
    tr.zmktbl {
      margin: 0;
    }
    td.zmktbl {
      border: 1px solid;
    }
    pre.zmktbl {
      margin: 4pt 1pt 1pt 13pt; 
      display: inline;
      font-family: monospace;
      font-size: 16px;
      font-weight: bold;
    }
    """

    return css, a, title


import contextlib
from airium import Airium


def airdoc(css, html, title):
    a = Airium()
    a("<!DOCTYPE html>")
    a.style(_t=css)
    with a.html():
        with a.head():
            a.meta(charset="utf-8")
            a.title(_t=title)
        with a.body():
            a.h3(_t=title)
            a.div(_t=html)

    return str(a)


skip_rows = set(range(5, 9)) | set(range(0x13, 0x1B))
ad = airdoc(*mktbl(Airium(), format_info_ocp_e3m2, cols=2, skip_rows=skip_rows))
with open("/tmp/aa.html", "w") as f:
    f.write(ad)
HTML(ad)

0x00 = 0_000_00 = 0.0 = 0.0,0x20 = 1_000_00 = -0.0 = -0.0
0x01 = 0_000_01 = +0b0.01*2^-2 = 0.0625,0x21 = 1_000_01 = -0b0.01*2^-2 = -0.0625
0x02 = 0_000_10 = +0b0.10*2^-2 = 0.125,0x22 = 1_000_10 = -0b0.10*2^-2 = -0.125
0x03 = 0_000_11 = +0b0.11*2^-2 = 0.1875,0x23 = 1_000_11 = -0b0.11*2^-2 = -0.1875
0x04 = 0_001_00 = +0b1.00*2^-2 = 0.25,0x24 = 1_001_00 = -0b1.00*2^-2 = -0.25
...,...
0x09 = 0_010_01 = +0b1.01*2^-1 = 0.625,0x29 = 1_010_01 = -0b1.01*2^-1 = -0.625
0x0a = 0_010_10 = +0b1.10*2^-1 = 0.75,0x2a = 1_010_10 = -0b1.10*2^-1 = -0.75
0x0b = 0_010_11 = +0b1.11*2^-1 = 0.875,0x2b = 1_010_11 = -0b1.11*2^-1 = -0.875
0x0c = 0_011_00 = +0b1.00*2^0 = 1.0,0x2c = 1_011_00 = -0b1.00*2^0 = -1.0
0x0d = 0_011_01 = +0b1.01*2^0 = 1.25,0x2d = 1_011_01 = -0b1.01*2^0 = -1.25


### OCP E2M1

This is a 4-bit format, without Inf or NaN

In [7]:
HTML(airdoc(*mktbl(Airium(), format_info_ocp_e2m1, cols=1, width=8, d=3)))

0x00 = 0_00_0 = 0.0 = 0.0
0x01 = 0_00_1 = +0b0.1*2^0 = 0.5
0x02 = 0_01_0 = +0b1.0*2^0 = 1.0
0x03 = 0_01_1 = +0b1.1*2^0 = 1.5
0x04 = 0_10_0 = +0b1.0*2^1 = 2.0
0x05 = 0_10_1 = +0b1.1*2^1 = 3.0
0x06 = 0_11_0 = +0b1.0*2^2 = 4.0
0x07 = 0_11_1 = +0b1.1*2^2 = 6.0
0x08 = 1_00_0 = -0.0 = -0.0
0x09 = 1_00_1 = -0b0.1*2^0 = -0.5
0x0a = 1_01_0 = -0b1.0*2^0 = -1.0


## A range of 4-bit formats

IEEE P3109 defines a range of 4-bit formats, varying precision, signedness, and domain.
Here we generate four tables for each precision.

Some observations about the formats: 
 - the p=1 formats are pure-exponent formats
 - the p=3 signed format is linear, but the p=3 unsigned format is floating
 - the p=4 unsigned format is linear, the p=4 signed format is identical to p=3

In [8]:
# Convert to PNG
import IPython


def render(fis, short=True):
    html = airdoc(*fourtables(fis, short=short, dark=False))
    display(HTML(html))


def fourtables(fis, **kw):
    a = airium.Airium()
    a("<!DOCTYPE html>")
    a.style(
        _t="""
        table {
          font-family: monospace;
          text-align: left;
          font-size: 1.3em;
        }
        .fourtables-td table {
          width: 100%;
        }
        td.zmktbl {
          border: 0px solid;
        }
        pre.zmktbl {
        }
        .fourtables-td tr:nth-child(odd) {
          background-color: #eee;
        }
        .fourtables-td tr:nth-child(even) {
          background-color: #ccc;
        }
        """
    )
    with a.div(style="width:1200px; background-color: #fff; color: black;"):
        with a.table(klass="fourtables-table"):
            for row in (0, 1):
                with a.tr(style="width:100%;"):
                    for fi in fis:
                        if row == 0:
                            a.th(klass="fourtables-th", _t=f"{fi}")
                        else:
                            with a.td(
                                klass="fourtables-td",
                                style="width:248px;",
                            ):
                                css, html, title = mktbl(
                                    a, fi, cols=1, width=8, d=4, **kw
                                )

    return css, str(a), ""


render([format_info_p3109(4, p) for p in range(1, 4)] + [format_info_ocp_e2m1])
render([format_info_p3109(4, p) for p in range(1, 4)], short=False)

In [9]:
for p in (1, 2, 3, 4):
    fis = [
        format_info_p3109(4, p, signedness, domain)
        for signedness in (Signedness.Signed, Signedness.Unsigned)
        for domain in (Domain.Extended, Domain.Finite)
    ]
    render(fis, short=False)

## Check k=5

In [10]:
for p in (1, 2, 3, 4, 5):
    fis = [
        format_info_p3109(5, p, signedness, domain)
        for signedness in (Signedness.Signed, Signedness.Unsigned)
        for domain in (Domain.Extended, Domain.Finite)
    ]
    render(fis, short=False)

### Now check unsigned vs signed, k=6

In [11]:
for p in (2, 3):
    fis = [
        format_info_p3109(6, p, signedness, domain)
        for signedness in (Signedness.Signed, Signedness.Unsigned)
        for domain in (Domain.Extended, Domain.Finite)
    ]
    render(fis, short=False)

### IEEE P3109 4-bit formats

The IEEE P3109 interim report describes a family of formats parameterized by K and P, in which three 4-bit formats are defined.

The p=2 format is similar to OCP E2M1, with inf and nan:

In [12]:
HTML(airdoc(*mktbl(Airium(), format_info_p3109(4, 2), cols=2, width=8, d=3)))

0x00 = 0_00_0 = 0.0 = 0.0,0x08 = 1_00_0 = nan = nan
0x01 = 0_00_1 = +0b0.1*2^-1 = 0.25,0x09 = 1_00_1 = -0b0.1*2^-1 = -0.25
0x02 = 0_01_0 = +0b1.0*2^-1 = 0.5,0x0a = 1_01_0 = -0b1.0*2^-1 = -0.5
0x03 = 0_01_1 = +0b1.1*2^-1 = 0.75,0x0b = 1_01_1 = -0b1.1*2^-1 = -0.75
0x04 = 0_10_0 = +0b1.0*2^0 = 1.0,0x0c = 1_10_0 = -0b1.0*2^0 = -1.0
0x05 = 0_10_1 = +0b1.1*2^0 = 1.5,0x0d = 1_10_1 = -0b1.1*2^0 = -1.5
0x06 = 0_11_0 = +0b1.0*2^1 = 2.0,0x0e = 1_11_0 = -0b1.0*2^1 = -2.0
0x07 = 0_11_1 = inf = inf,0x0f = 1_11_1 = -inf = -inf


While the p=1 format is a "pure exponential" format with values 2^-2 to 2^3:

In [13]:
HTML(airdoc(*mktbl(Airium(), format_info_p3109(4, 1), cols=2, width=8, d=3)))

0x00 = 0_000_ = 0.0 = 0.0,0x08 = 1_000_ = nan = nan
0x01 = 0_001_ = +0b1.0*2^-3 = 0.125,0x09 = 1_001_ = -0b1.0*2^-3 = -0.125
0x02 = 0_010_ = +0b1.0*2^-2 = 0.25,0x0a = 1_010_ = -0b1.0*2^-2 = -0.25
0x03 = 0_011_ = +0b1.0*2^-1 = 0.5,0x0b = 1_011_ = -0b1.0*2^-1 = -0.5
0x04 = 0_100_ = +0b1.0*2^0 = 1.0,0x0c = 1_100_ = -0b1.0*2^0 = -1.0
0x05 = 0_101_ = +0b1.0*2^1 = 2.0,0x0d = 1_101_ = -0b1.0*2^1 = -2.0
0x06 = 0_110_ = +0b1.0*2^2 = 4.0,0x0e = 1_110_ = -0b1.0*2^2 = -4.0
0x07 = 0_111_ = inf = inf,0x0f = 1_111_ = -inf = -inf


And p=3, a linear format with values 0.25 * range(7)

In [14]:
HTML(airdoc(*mktbl(Airium(), format_info_p3109(4, 3), cols=2, width=8, d=3)))

0x00 = 0_0_00 = 0.0 = 0.0,0x08 = 1_0_00 = nan = nan
0x01 = 0_0_01 = +0b0.01*2^0 = 0.25,0x09 = 1_0_01 = -0b0.01*2^0 = -0.25
0x02 = 0_0_10 = +0b0.10*2^0 = 0.5,0x0a = 1_0_10 = -0b0.10*2^0 = -0.5
0x03 = 0_0_11 = +0b0.11*2^0 = 0.75,0x0b = 1_0_11 = -0b0.11*2^0 = -0.75
0x04 = 0_1_00 = +0b1.00*2^0 = 1.0,0x0c = 1_1_00 = -0b1.00*2^0 = -1.0
0x05 = 0_1_01 = +0b1.01*2^0 = 1.25,0x0d = 1_1_01 = -0b1.01*2^0 = -1.25
0x06 = 0_1_10 = +0b1.10*2^0 = 1.5,0x0e = 1_1_10 = -0b1.10*2^0 = -1.5
0x07 = 0_1_11 = inf = inf,0x0f = 1_1_11 = -inf = -inf


### OCP E2M3

This 6-bit format has 32 values, with no `NaN` or `Inf`, but does have `-0`.
The positive subnormals are the linear ramp of eighths: [n/8 for n in 1:7].

One might describe the format in text as:

> zero to one by eighths, two to four by quarters, four to eight by halves

where "to" is open-ended, or "to" is not "thru".

In [15]:
HTML(airdoc(*mktbl(Airium(), format_info_ocp_e2m3, cols=2, width=8, d=3)))

0x00 = 0_00_000 = 0.0 = 0.0,0x20 = 1_00_000 = -0.0 = -0.0
0x01 = 0_00_001 = +0b0.001*2^0 = 0.125,0x21 = 1_00_001 = -0b0.001*2^0 = -0.125
0x02 = 0_00_010 = +0b0.010*2^0 = 0.25,0x22 = 1_00_010 = -0b0.010*2^0 = -0.25
0x03 = 0_00_011 = +0b0.011*2^0 = 0.375,0x23 = 1_00_011 = -0b0.011*2^0 = -0.375
0x04 = 0_00_100 = +0b0.100*2^0 = 0.5,0x24 = 1_00_100 = -0b0.100*2^0 = -0.5
0x05 = 0_00_101 = +0b0.101*2^0 = 0.625,0x25 = 1_00_101 = -0b0.101*2^0 = -0.625
0x06 = 0_00_110 = +0b0.110*2^0 = 0.75,0x26 = 1_00_110 = -0b0.110*2^0 = -0.75
0x07 = 0_00_111 = +0b0.111*2^0 = 0.875,0x27 = 1_00_111 = -0b0.111*2^0 = -0.875
0x08 = 0_01_000 = +0b1.000*2^0 = 1.0,0x28 = 1_01_000 = -0b1.000*2^0 = -1.0
0x09 = 0_01_001 = +0b1.001*2^0 = 1.125,0x29 = 1_01_001 = -0b1.001*2^0 = -1.125
0x0a = 0_01_010 = +0b1.010*2^0 = 1.25,0x2a = 1_01_010 = -0b1.010*2^0 = -1.25


### OCP Formats: E5M2, E4M3

In [16]:
# HTML(mktbl(format_info_ocp_e5m2, cols=4, skip_rows=(0x10, 0x30), vs_width=8, vs_d=5))
HTML(
    airdoc(
        *mktbl(
            Airium(),
            format_info_ocp_e5m2,
            skip_rows=range(0x10, 0x30),
            cols=4,
            width=8,
            d=3,
        )
    )
)

0x00 = 0_00000_00 = 0.0 = 0.0,0x40 = 0_10000_00 = +0b1.00*2^1 = 2.0,0x80 = 1_00000_00 = -0.0 = -0.0,0xc0 = 1_10000_00 = -0b1.00*2^1 = -2.0
0x01 = 0_00000_01 = +0b0.01*2^-14 = ~1.53e-05,0x41 = 0_10000_01 = +0b1.01*2^1 = 2.5,0x81 = 1_00000_01 = -0b0.01*2^-14 = ~-1.53e-05,0xc1 = 1_10000_01 = -0b1.01*2^1 = -2.5
0x02 = 0_00000_10 = +0b0.10*2^-14 = ~3.05e-05,0x42 = 0_10000_10 = +0b1.10*2^1 = 3.0,0x82 = 1_00000_10 = -0b0.10*2^-14 = ~-3.05e-05,0xc2 = 1_10000_10 = -0b1.10*2^1 = -3.0
0x03 = 0_00000_11 = +0b0.11*2^-14 = ~4.58e-05,0x43 = 0_10000_11 = +0b1.11*2^1 = 3.5,0x83 = 1_00000_11 = -0b0.11*2^-14 = ~-4.58e-05,0xc3 = 1_10000_11 = -0b1.11*2^1 = -3.5
0x04 = 0_00001_00 = +0b1.00*2^-14 = ~6.1e-05,0x44 = 0_10001_00 = +0b1.00*2^2 = 4.0,0x84 = 1_00001_00 = -0b1.00*2^-14 = ~-6.1e-05,0xc4 = 1_10001_00 = -0b1.00*2^2 = -4.0
0x05 = 0_00001_01 = +0b1.01*2^-14 = ~7.63e-05,0x45 = 0_10001_01 = +0b1.01*2^2 = 5.0,0x85 = 1_00001_01 = -0b1.01*2^-14 = ~-7.63e-05,0xc5 = 1_10001_01 = -0b1.01*2^2 = -5.0
0x06 = 0_00001_10 = +0b1.10*2^-14 = ~9.16e-05,0x46 = 0_10001_10 = +0b1.10*2^2 = 6.0,0x86 = 1_00001_10 = -0b1.10*2^-14 = ~-9.16e-05,0xc6 = 1_10001_10 = -0b1.10*2^2 = -6.0
0x07 = 0_00001_11 = +0b1.11*2^-14 = ~0.000,0x47 = 0_10001_11 = +0b1.11*2^2 = 7.0,0x87 = 1_00001_11 = -0b1.11*2^-14 = ~-0.000,0xc7 = 1_10001_11 = -0b1.11*2^2 = -7.0
0x08 = 0_00010_00 = +0b1.00*2^-13 = ~0.000,0x48 = 0_10010_00 = +0b1.00*2^3 = 8.0,0x88 = 1_00010_00 = -0b1.00*2^-13 = ~-0.000,0xc8 = 1_10010_00 = -0b1.00*2^3 = -8.0
0x09 = 0_00010_01 = +0b1.01*2^-13 = ~0.000,0x49 = 0_10010_01 = +0b1.01*2^3 = 10.0,0x89 = 1_00010_01 = -0b1.01*2^-13 = ~-0.000,0xc9 = 1_10010_01 = -0b1.01*2^3 = -10.0
0x0a = 0_00010_10 = +0b1.10*2^-13 = ~0.000,0x4a = 0_10010_10 = +0b1.10*2^3 = 12.0,0x8a = 1_00010_10 = -0b1.10*2^-13 = ~-0.000,0xca = 1_10010_10 = -0b1.10*2^3 = -12.0


In [17]:
HTML(
    airdoc(
        *mktbl(
            Airium(),
            format_info_ocp_e4m3,
            skip_rows=range(0x10, 0x30),
            cols=4,
            width=8,
            d=3,
        )
    )
)

0x00 = 0_0000_000 = 0.0 = 0.0,0x40 = 0_1000_000 = +0b1.000*2^1 = 2.0,0x80 = 1_0000_000 = -0.0 = -0.0,0xc0 = 1_1000_000 = -0b1.000*2^1 = -2.0
0x01 = 0_0000_001 = +0b0.001*2^-6 = ~0.002,0x41 = 0_1000_001 = +0b1.001*2^1 = 2.25,0x81 = 1_0000_001 = -0b0.001*2^-6 = ~-0.002,0xc1 = 1_1000_001 = -0b1.001*2^1 = -2.25
0x02 = 0_0000_010 = +0b0.010*2^-6 = ~0.004,0x42 = 0_1000_010 = +0b1.010*2^1 = 2.5,0x82 = 1_0000_010 = -0b0.010*2^-6 = ~-0.004,0xc2 = 1_1000_010 = -0b1.010*2^1 = -2.5
0x03 = 0_0000_011 = +0b0.011*2^-6 = ~0.006,0x43 = 0_1000_011 = +0b1.011*2^1 = 2.75,0x83 = 1_0000_011 = -0b0.011*2^-6 = ~-0.006,0xc3 = 1_1000_011 = -0b1.011*2^1 = -2.75
0x04 = 0_0000_100 = +0b0.100*2^-6 = ~0.008,0x44 = 0_1000_100 = +0b1.100*2^1 = 3.0,0x84 = 1_0000_100 = -0b0.100*2^-6 = ~-0.008,0xc4 = 1_1000_100 = -0b1.100*2^1 = -3.0
0x05 = 0_0000_101 = +0b0.101*2^-6 = ~0.010,0x45 = 0_1000_101 = +0b1.101*2^1 = 3.25,0x85 = 1_0000_101 = -0b0.101*2^-6 = ~-0.010,0xc5 = 1_1000_101 = -0b1.101*2^1 = -3.25
0x06 = 0_0000_110 = +0b0.110*2^-6 = ~0.012,0x46 = 0_1000_110 = +0b1.110*2^1 = 3.5,0x86 = 1_0000_110 = -0b0.110*2^-6 = ~-0.012,0xc6 = 1_1000_110 = -0b1.110*2^1 = -3.5
0x07 = 0_0000_111 = +0b0.111*2^-6 = ~0.014,0x47 = 0_1000_111 = +0b1.111*2^1 = 3.75,0x87 = 1_0000_111 = -0b0.111*2^-6 = ~-0.014,0xc7 = 1_1000_111 = -0b1.111*2^1 = -3.75
0x08 = 0_0001_000 = +0b1.000*2^-6 = 0.015625,0x48 = 0_1001_000 = +0b1.000*2^2 = 4.0,0x88 = 1_0001_000 = -0b1.000*2^-6 = ~-0.016,0xc8 = 1_1001_000 = -0b1.000*2^2 = -4.0
0x09 = 0_0001_001 = +0b1.001*2^-6 = ~0.018,0x49 = 0_1001_001 = +0b1.001*2^2 = 4.5,0x89 = 1_0001_001 = -0b1.001*2^-6 = ~-0.018,0xc9 = 1_1001_001 = -0b1.001*2^2 = -4.5
0x0a = 0_0001_010 = +0b1.010*2^-6 = ~0.020,0x4a = 0_1001_010 = +0b1.010*2^2 = 5.0,0x8a = 1_0001_010 = -0b1.010*2^-6 = ~-0.020,0xca = 1_1001_010 = -0b1.010*2^2 = -5.0


In [18]:
HTML(
    airdoc(
        *mktbl(
            Airium(),
            format_info_ocp_e8m0,
            skip_rows=range(0x10, 0x30),
            cols=4,
            width=8,
            d=3,
        )
    )
)

0x00 = 00000000_ = +0b1.0*2^-127 = ~5.88e-39,0x40 = 01000000_ = +0b1.0*2^-63 = ~1.08e-19,0x80 = 10000000_ = +0b1.0*2^1 = 2.0,0xc0 = 11000000_ = +0b1.0*2^65 = ~3.69e+19
0x01 = 00000001_ = +0b1.0*2^-126 = ~1.18e-38,0x41 = 01000001_ = +0b1.0*2^-62 = ~2.17e-19,0x81 = 10000001_ = +0b1.0*2^2 = 4.0,0xc1 = 11000001_ = +0b1.0*2^66 = ~7.38e+19
0x02 = 00000010_ = +0b1.0*2^-125 = ~2.35e-38,0x42 = 01000010_ = +0b1.0*2^-61 = ~4.34e-19,0x82 = 10000010_ = +0b1.0*2^3 = 8.0,0xc2 = 11000010_ = +0b1.0*2^67 = ~1.48e+20
0x03 = 00000011_ = +0b1.0*2^-124 = ~4.7e-38,0x43 = 01000011_ = +0b1.0*2^-60 = ~8.67e-19,0x83 = 10000011_ = +0b1.0*2^4 = 16.0,0xc3 = 11000011_ = +0b1.0*2^68 = ~2.95e+20
0x04 = 00000100_ = +0b1.0*2^-123 = ~9.4e-38,0x44 = 01000100_ = +0b1.0*2^-59 = ~1.73e-18,0x84 = 10000100_ = +0b1.0*2^5 = 32.0,0xc4 = 11000100_ = +0b1.0*2^69 = ~5.9e+20
0x05 = 00000101_ = +0b1.0*2^-122 = ~1.88e-37,0x45 = 01000101_ = +0b1.0*2^-58 = ~3.47e-18,0x85 = 10000101_ = +0b1.0*2^6 = 64.0,0xc5 = 11000101_ = +0b1.0*2^70 = ~1.18e+21
0x06 = 00000110_ = +0b1.0*2^-121 = ~3.76e-37,0x46 = 01000110_ = +0b1.0*2^-57 = ~6.94e-18,0x86 = 10000110_ = +0b1.0*2^7 = 128.0,0xc6 = 11000110_ = +0b1.0*2^71 = ~2.36e+21
0x07 = 00000111_ = +0b1.0*2^-120 = ~7.52e-37,0x47 = 01000111_ = +0b1.0*2^-56 = ~1.39e-17,0x87 = 10000111_ = +0b1.0*2^8 = 256.0,0xc7 = 11000111_ = +0b1.0*2^72 = ~4.72e+21
0x08 = 00001000_ = +0b1.0*2^-119 = ~1.5e-36,0x48 = 01001000_ = +0b1.0*2^-55 = ~2.78e-17,0x88 = 10001000_ = +0b1.0*2^9 = 512.0,0xc8 = 11001000_ = +0b1.0*2^73 = ~9.44e+21
0x09 = 00001001_ = +0b1.0*2^-118 = ~3.01e-36,0x49 = 01001001_ = +0b1.0*2^-54 = ~5.55e-17,0x89 = 10001001_ = +0b1.0*2^10 = 1024.0,0xc9 = 11001001_ = +0b1.0*2^74 = ~1.89e+22
0x0a = 00001010_ = +0b1.0*2^-117 = ~6.02e-36,0x4a = 01001010_ = +0b1.0*2^-53 = ~1.11e-16,0x8a = 10001010_ = +0b1.0*2^11 = 2048.0,0xca = 11001010_ = +0b1.0*2^75 = ~3.78e+22


### IEEE WG P3109 KpP formats

We choose just one example: `p3109(k=8, p=3)`, which has the same number of exponent bits as OCP E5 

In [19]:
HTML(
    airdoc(
        *mktbl(
            Airium(),
            format_info_p3109(8, 1, Signedness.Unsigned, Domain.Finite),
            skip_rows=range(0x10, 0x30),
            cols=4,
            width=8,
            d=3,
        )
    )
)

0x00 = 00000000_ = 0.0 = 0.0,0x40 = 01000000_ = +0b1.0*2^-64 = ~5.42e-20,0x80 = 10000000_ = +0b1.0*2^0 = 1.0,0xc0 = 11000000_ = +0b1.0*2^64 = ~1.84e+19
0x01 = 00000001_ = +0b1.0*2^-127 = ~5.88e-39,0x41 = 01000001_ = +0b1.0*2^-63 = ~1.08e-19,0x81 = 10000001_ = +0b1.0*2^1 = 2.0,0xc1 = 11000001_ = +0b1.0*2^65 = ~3.69e+19
0x02 = 00000010_ = +0b1.0*2^-126 = ~1.18e-38,0x42 = 01000010_ = +0b1.0*2^-62 = ~2.17e-19,0x82 = 10000010_ = +0b1.0*2^2 = 4.0,0xc2 = 11000010_ = +0b1.0*2^66 = ~7.38e+19
0x03 = 00000011_ = +0b1.0*2^-125 = ~2.35e-38,0x43 = 01000011_ = +0b1.0*2^-61 = ~4.34e-19,0x83 = 10000011_ = +0b1.0*2^3 = 8.0,0xc3 = 11000011_ = +0b1.0*2^67 = ~1.48e+20
0x04 = 00000100_ = +0b1.0*2^-124 = ~4.7e-38,0x44 = 01000100_ = +0b1.0*2^-60 = ~8.67e-19,0x84 = 10000100_ = +0b1.0*2^4 = 16.0,0xc4 = 11000100_ = +0b1.0*2^68 = ~2.95e+20
0x05 = 00000101_ = +0b1.0*2^-123 = ~9.4e-38,0x45 = 01000101_ = +0b1.0*2^-59 = ~1.73e-18,0x85 = 10000101_ = +0b1.0*2^5 = 32.0,0xc5 = 11000101_ = +0b1.0*2^69 = ~5.9e+20
0x06 = 00000110_ = +0b1.0*2^-122 = ~1.88e-37,0x46 = 01000110_ = +0b1.0*2^-58 = ~3.47e-18,0x86 = 10000110_ = +0b1.0*2^6 = 64.0,0xc6 = 11000110_ = +0b1.0*2^70 = ~1.18e+21
0x07 = 00000111_ = +0b1.0*2^-121 = ~3.76e-37,0x47 = 01000111_ = +0b1.0*2^-57 = ~6.94e-18,0x87 = 10000111_ = +0b1.0*2^7 = 128.0,0xc7 = 11000111_ = +0b1.0*2^71 = ~2.36e+21
0x08 = 00001000_ = +0b1.0*2^-120 = ~7.52e-37,0x48 = 01001000_ = +0b1.0*2^-56 = ~1.39e-17,0x88 = 10001000_ = +0b1.0*2^8 = 256.0,0xc8 = 11001000_ = +0b1.0*2^72 = ~4.72e+21
0x09 = 00001001_ = +0b1.0*2^-119 = ~1.5e-36,0x49 = 01001001_ = +0b1.0*2^-55 = ~2.78e-17,0x89 = 10001001_ = +0b1.0*2^9 = 512.0,0xc9 = 11001001_ = +0b1.0*2^73 = ~9.44e+21
0x0a = 00001010_ = +0b1.0*2^-118 = ~3.01e-36,0x4a = 01001010_ = +0b1.0*2^-54 = ~5.55e-17,0x8a = 10001010_ = +0b1.0*2^10 = 1024.0,0xca = 11001010_ = +0b1.0*2^74 = ~1.89e+22


### Some tiny tiny formats

And finally, some tiny tiny formats.  We will take a finite P3109 format.

For p=1, we get as usual, a pure-exponential format with range 2^-1 to 2^1:

In [20]:
p3109_3p1f = format_info_p3109(3, 1, Signedness.Signed, Domain.Finite)
HTML(airdoc(*mktbl(Airium(), p3109_3p1f, cols=2, width=8, d=3)))

0x00 = 0_00_ = 0.0 = 0.0,0x04 = 1_00_ = nan = nan
0x01 = 0_01_ = +0b1.0*2^-1 = 0.5,0x05 = 1_01_ = -0b1.0*2^-1 = -0.5
0x02 = 0_10_ = +0b1.0*2^0 = 1.0,0x06 = 1_10_ = -0b1.0*2^0 = -1.0
0x03 = 0_11_ = +0b1.0*2^1 = 2.0,0x07 = 1_11_ = -0b1.0*2^1 = -2.0


And for p=2, we get, as usual for p=k-1, a purely linear format.  In this case, with values (0.5, 1.0, 1.5). Again as usual, 1.0 is in the middle of the range.

In [21]:
p3109_3p1f = format_info_p3109(3, 2, Signedness.Signed, Domain.Finite)
HTML(airdoc(*mktbl(Airium(), p3109_3p1f, cols=2, width=8, d=3)))

0x00 = 0_0_0 = 0.0 = 0.0,0x04 = 1_0_0 = nan = nan
0x01 = 0_0_1 = +0b0.1*2^0 = 0.5,0x05 = 1_0_1 = -0b0.1*2^0 = -0.5
0x02 = 0_1_0 = +0b1.0*2^0 = 1.0,0x06 = 1_1_0 = -0b1.0*2^0 = -1.0
0x03 = 0_1_1 = +0b1.1*2^0 = 1.5,0x07 = 1_1_1 = -0b1.1*2^0 = -1.5
